In [ ]:
import os
import sys
from math import ceil
import shutil
import IPython
import ipywidgets
from PIL import Image, ImageFont, ImageDraw, ExifTags
from time import sleep

In [ ]:
class ProcessItems:
    def __init__(self,
                 SEPARATOR=' - ',
                 PRICE_PREFIX=u"\u00A3", # £ pound sign (GBP)
                 PRICE_SUFFIX='',
                 PRICE_SALE=100,
                 PRICE_SCALEFACTOR=1,
                 PRICE_TONEAREST=1,
                 PRICE_FILE_PATH='pricing.txt',
                 PRICE_FILE_PATH_OLD='pricing_old.txt',
                 SOLD_TAG='SOLD',
                 SOLD_SHOW=True,
                 NFS_TAG='NFS',
                 NFS_SHOW=True,
                 IGNORE_TAG='IGNORE',
                 FINISH_TAG='FINISH',
                 TEXT_DROPCOLOUR=(0, 0, 0),
                 TEXT_COLOUR=(255, 255, 255),
                 TEXT_SIZE=40,
                 TEXT_FONT=None
                ):
        
        TEXT_FONT = "FreeMono.ttf" if os.name == "posix" else "arial.ttf" # default font based on OS (windows and linux only - if using a MAC try Arial.ttf)
        
        # Configuration parameters 
        self.config = {
            'SEPARATOR' : SEPARATOR, # what separates file names from their prices in the pricing file
            'PRICE_PREFIX' : PRICE_PREFIX, # what to prepend to the prices (usually a currency symbol)
            'PRICE_SUFFIX' : PRICE_SUFFIX, # what to append to the prices (usually nothing)
            'PRICE_SALE' : PRICE_SALE, # percentage of original price (100 = no sale, 120 = 20% up, 80 = 20% off)
            'PRICE_SCALEFACTOR' : PRICE_SCALEFACTOR, # allow scaling of price for different currencies and conversation rates!
            'PRICE_TONEAREST' : PRICE_TONEAREST, # round to the nearest.. (1 rounds to nearest integer, 0.5 rounds to nearest 0.5, 1, 1.5 etc) - always rounds up
            'PRICE_FILE_PATH' : PRICE_FILE_PATH, # the file to look for in the items directory to store price information
            'PRICE_FILE_PATH_OLD' : PRICE_FILE_PATH_OLD, # same as above, but for the backup file if the original is ever overwritten 
            'SOLD_TAG' : SOLD_TAG, # when looking at prices, if the SOLD_TAG word is found, then the item is considered to be sold and no longer for sale
            'SOLD_SHOW' : SOLD_SHOW, # whether or not to process items which have sold
            'NFS_TAG' : NFS_TAG, # when looking at prices, if the NFS_TAG word is found, the item is considered to be not for sale
            'NFS_SHOW' : NFS_SHOW, # whether or not to process items which are NFS
            'IGNORE_TAG' : IGNORE_TAG, # when looking at prices, if the IGNORE_TAG word is found, the item is ignored and not processed
            'FINISH_TAG' : FINISH_TAG, # when looking at prices, if the FINISH_TAG word is found, then abort pricing and save current state
            'TEXT_DROPCOLOUR' : TEXT_DROPCOLOUR, # colour of the dropshadow for the price tag
            'TEXT_COLOUR' : TEXT_COLOUR, # colour of the price tag (text)
            'TEXT_SIZE' : TEXT_SIZE, # text sixe will expand to TEXT_SIZE% of the image width (30-50% is a good range)
            'TEXT_FONT' : TEXT_FONT # font to use for price tag
        }
        
        # Find the directory with items and the pricing file
        self.loadItemsDirectory()
        
        # Find and create (if necessary) the directory for the processed items (i.e. with price tags)
        self.loadProcessedDirectory()
        
        # Find all the items in the items directory and store in memory
        self.loadItems()
        
        # Find all the items referenced in the pricing file and store in memory
        self.loadPricing()
        
        # Cross reference the two lists above and ensure yet-to-be-referenced items are priced, and that there are no priced items which do not exist in the items directory
        self.askForMissingPrices()
        
        # Add the price tags
        self.processPrices()
        
        # Print to console that the processing has been complete
        self.endWithMessage()
        
    def loadItemsDirectory(self):
        
        ### Find the directory with items and the pricing file
        
        print("You are now in %s"%(os.getcwd()))
        item_path = self.haltProcessing("Directory where your items exist: ")
        if item_path != "" and os.path.isdir(item_path): # is a valid directory
            item_path = os.path.join(item_path, '')
            self.item_path = item_path
            print("Item directory: %s"%(os.path.join(os.getcwd(), item_path)))
        else:
            print("Error: please select a valid directory")
            self.loadItemsDirectory()
            
    def loadProcessedDirectory(self):
        
        ### Find and create (if necessary) the directory for the processed items (i.e. with price tags)
        
        output_path = self.haltProcessing("Directory for your processed items: ")
        if output_path != "": # not empty
            output_path = os.path.join(output_path, '')
            if not os.path.isdir(output_path): # if directory doesn't exist, ask to create it
                create_new = self.haltProcessing("Directory does not exist, create it? [y/n] ")
                if create_new == "y": # create the directory
                    os.makedirs(output_path)
                    
            # this will be true unless the user did not want to create the non-existing directory
            if os.path.isdir(output_path):
                if output_path != self.item_path: # avoid overwriting of original item images
                    if len(os.listdir(output_path)) == 0: # output directory must be empty
                        self.output_path = output_path
                        print("Processed directory: %s"%(os.path.join(os.getcwd(), output_path)))
                        return True # all checks passed
                    print("Error: The processed directory must be empty")
                    print(os.listdir(output_path))
                else:
                    print("Error: processed directory cannot be the same as the items directory")
        else:
            print("Error: please select a valid directory")
            
        # rerun function unless all checks have passed
        self.loadProcessedDirectory()
        
    def loadItems(self):
        
        ### Find all the items in the items directory and store in memory
        
        self.items = set()
        self.excluded_items = set()
        
        for f in os.listdir(self.item_path): # loop through all files in item directory
            if os.path.isfile(os.path.join(self.item_path, f)): # ignore sub-directories
                if f.endswith(('.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG')): # is an image
                    self.items.add(f) # add to 'memory' of items
                elif f != 'pricing.txt' and f != 'pricing_old.txt': # as long as not a pricing data file
                    self.excluded_items.add(f) # add to 'memory' of items which are not valid images (i.e. will not be processed)
                    
        print("%d items loaded"%(len(self.items)))
        
        if len(self.excluded_items) > 0: # if there are invalid files, alert the user before continuing (possibly wrong directory selected)
            print("%d items ignored"%(len(self.excluded_items)))
            print(" > %s"%(', '.join(self.excluded_items)))
            if self.haltProcessing("Continue? [y/n] ") != "y":
                sys.exit() # Abort
                        
    def loadPricing(self):
        
        ### Find all the items referenced in the pricing file and store in memory
        
        self.pricing_file_path = os.path.join(self.item_path, self.config['PRICE_FILE_PATH'])
        self.priced_items = {}
        
        if os.path.isfile(self.pricing_file_path): # pricing data file exists
            if self.haltProcessing("Pricing file already exists, do you want to use this data? [y/n] ") != 'n':
                with open(self.pricing_file_path) as pricing_file:
                    pricing_file_contents = pricing_file.read().splitlines() # Read in pricing data from file line-by-line
                for item in pricing_file_contents:
                    item_file, price = item.split(self.config['SEPARATOR']) # 'ITEM FILE NAME' SEPARATOR 'PRICE'
                    self.priced_items[item_file] = price # store in memory
                return True
            else:
                # Backup original pricing data file for later reference (only 1 file can be backed up at a time)
                # Later backups replace original backups
                shutil.move(self.pricing_file_path, os.path.join(self.item_path, self.config['PRICE_FILE_PATH_OLD']))
                print("Pricing file backed up in %s"%(self.config['PRICE_FILE_PATH_OLD']))
        
        # this is run if pricing data file doesn't exist, or if it does but would prefer to use a new one
        print("Creating a new pricing file")
        open(self.pricing_file_path, 'w') # create new, empty pricing data file
            
    def askForMissingPrices(self, force_exit=False):
        
        ### Cross reference the two lists above and ensure yet-to-be-referenced items are priced, and that there are no priced items which do not exist in the items directory
        
        items_to_process = self.items # set
        items_with_prices = set(self.priced_items.keys()) # set
        items_needing_prices = items_to_process - items_with_prices # set of items which have yet to be priced in the pricing file
        items_dont_exist = items_with_prices - items_to_process # set of items which have been priced, but dont exist in the items directory
        
        if len(items_dont_exist) > 0: # Some items in the pricing data file do not exist in the items directory
            
            if force_exit: # this method has been called for the second time but there are still items which shouldn't exist..
                print("There was an error when removing the unidentified items")
                sys.exit()
            
            # 
            action = self.haltProcessing("%d items in the pricing file do not exist.\n %s \nRemove these from the file? [y/n] "%(len(items_dont_exist), '\n'.join(items_dont_exist)))
            if action == 'y':
                
                shutil.move(self.pricing_file_path, os.path.join(self.item_path, self.config['PRICE_FILE_PATH_OLD']))
                print("Pricing file backed up in %s"%(self.config['PRICE_FILE_PATH_OLD']))
                
                # remove from the priced items those that don't exist
                # then rerun the function 
                for item in items_dont_exist:
                    del self.priced_items[item] # delete from dictionary
                    
                return self.askForMissingPrices(force_exit=True)
            else:
                # If invalid items are not removed from the file we will stop processing to prevent further errors
                self.haltProcessing("Processing has ended. Please make sure the items and pricing file match up!")
                sys.exit() # abort!
                
        if len(items_needing_prices) > 0: # some items in the items directory have not yet been priced
            print("\nPlease enter the price for the following items (no currency symbols)")
            print("If the item is sold, type '%s'"%(self.config['SOLD_TAG']))
            print("If the item is not for sale, type '%s'"%(self.config['NFS_TAG']))
            print("To ignore the item, type '%s'"%(self.config['IGNORE_TAG']))
            print("To end the pricing and save the current progress, type '%s'"%(self.config['FINISH_TAG']))
           
            for item in sorted(items_needing_prices): # For each item which hasn't yet been priced
                item_path = os.path.join(self.item_path, item)
                image = Image.open(item_path)
                image = self.fixEXIFData(image)
                image.thumbnail((400, 400), Image.ANTIALIAS)
                IPython.display.display(image) # Show item image
                price = False
                while price != self.config['SOLD_TAG'] and price != self.config['NFS_TAG'] and price != self.config['IGNORE_TAG'] and price != self.config['FINISH_TAG'] and not isinstance(price, float): # Until user enters a valid price (number or SOLD_TAG or NFS_TAG)
                    price = self.haltProcessing("Price: ")
                    if price == self.config['FINISH_TAG']: # user has decided to stop pricing - save status for now
                        break
                    if price != self.config['SOLD_TAG'] and price != self.config['NFS_TAG'] and price != self.config['IGNORE_TAG']:
                        try:
                            price = float(price) # then convert string to float
                        except ValueError as e:
                            continue # price is not valid, ask again
                            
                IPython.display.clear_output() # Remove the image from display
                
                if price == self.config['FINISH_TAG']: # break out of loop if asked to finish
                    break
                else: # as long as we're not ignoring this item, then assign the price
                    self.priced_items[item] = price # Assign the price to the item
                    self.savePricesForFuture() # Update pricing data file with updated prices (save here incase of error so no lost data)
    
    def processPrices(self):
        
        ### Add the price tags to the image (as a copy in the output folder)
        
        self.fontsize = 1 # store for later use
        
        items_to_process = self.items # set
        items_with_prices = set(self.priced_items.keys()) # set
        items_needing_prices = items_to_process - items_with_prices # set of items which have yet to be priced in the pricing file
        items_dont_exist = items_with_prices - items_to_process # set of items which have been priced, but dont exist in the items directory
       
        self.savePricesForFuture() # Update pricing data file with updated prices
    
        if len(items_needing_prices) > 0: # user must have input FINISH_TAG during pricing
            
            # ask user if they want to process those that are ready or not
            process = self.haltProcessing("Not all items have been priced, do you want to process those that are ready? [y/n] ")
            if process != 'y':
                print("Processing terminated")
                sys.exit()
            
        IPython.display.clear_output(wait=True) # remove display output
        progress_bar = ipywidgets.FloatProgress(min=0, max=len(items_with_prices)) # create a progress bar (image manipulation can take a while)
        IPython.display.display(progress_bar) # display the progress bar
    
        for item, price in self.priced_items.iteritems(): # for each item
            if price == self.config['IGNORE_TAG']: # if this item is to be ignored
                progress_bar.value += 1
                continue
                
            if price == self.config['SOLD_TAG'] or price == self.config['NFS_TAG']: # if either SOLD or NFS
                if price == self.config['SOLD_TAG'] and not self.config['SOLD_SHOW']: # ignore item if SOLD and SOLD_SHOW is False
                    progress_bar.value += 1
                    continue
                elif price == self.config['NFS_TAG'] and not self.config['NFS_SHOW']: # ignore item if NFS and NFS_SHOW is False
                    progress_bar.value += 1
                    continue
            else:            
                price = self.beautifyPrice(price) # Converts price by applying sale percentage, currency conversion and also prefix and suffix (see configuration options)
           
            self.createNewImage(item, price) # Copy the image to the output folder and add the price tag
            progress_bar.value += 1 # Update progress bar
            
    def savePricesForFuture(self):
        
        ### Update pricing data file with updated prices
        
        items = []
        for item, price in self.priced_items.iteritems():
            items.append("%s%s%s"%(item, self.config['SEPARATOR'], price)) # file formatting
            
        # write to file
        with open(self.pricing_file_path, mode='wt') as pricing_file:
            pricing_file.write('\n'.join(items))
            
    def createNewImage(self, item, price):
        
        ### Copy the image to the output folder and add the price tag
        
        original_path = os.path.join(self.item_path, item)
        processed_path = os.path.join(self.output_path, item)
        image = Image.open(original_path)
        image = self.fixEXIFData(image)
        
        draw = ImageDraw.Draw(image)
        fontsize = self.fontsize
        font = ImageFont.truetype(self.config['TEXT_FONT'], fontsize)
        target_size = self.config['TEXT_SIZE'] * image.size[0] / 100
        
        ## When the first image is being created, we start at a fontsize of 1 and incrementally increase
        ## until we are just above the target size. Final fontsize is stored in memory.
        ## For following runs, we use the stored font size, and if its too big for the new image (arbitrary size)
        ## then decrease it until we fall in the range
        ## Then increase it once (second while loop) to ensure it is just slightly bigger than the target size
        ## in order to maintain consistency with the first image
        
        while font.getsize(price)[0] >= target_size:
            fontsize -= 1
            font = ImageFont.truetype(self.config['TEXT_FONT'], fontsize)
        
        while font.getsize(price)[0] < target_size:
            fontsize += 1
            font = ImageFont.truetype(self.config['TEXT_FONT'], fontsize)
            
        self.fontsize = fontsize # store for next run
        
        w, h = font.getsize(price) # width, height of text
        W, H = image.size # width, height of image
        from_left = (W - w) / 2 # center horizontally
        from_top = 0.8 * H - h # 80% from top
        draw.text((from_left, from_top + 2), price, self.config['TEXT_DROPCOLOUR'], font=font) # dropshadow
        draw.text((from_left, from_top), price, self.config['TEXT_COLOUR'], font=font)
        image.save(processed_path) # save image in output folder
    
    def fixEXIFData(self, image):
        if hasattr(image, '_getexif'): # only present in JPEGs
            for orientation in ExifTags.TAGS.keys(): 
                if ExifTags.TAGS[orientation]=='Orientation':
                    break 
            e = image._getexif()       # returns None if no EXIF data
            if e is not None:
                exif=dict(e.items())
                orientation = exif[orientation] 

                if orientation == 3:   image = image.transpose(Image.ROTATE_180)
                elif orientation == 6: image = image.transpose(Image.ROTATE_270)
                elif orientation == 8: image = image.transpose(Image.ROTATE_90)
        return image
    
    def beautifyPrice(self, price):
        
        ### Converts price by applying sale percentage, currency conversion and also prefix and suffix (see configuration options)
        
        if price == self.config['SOLD_TAG'] or price == self.config['NFS_TAG']:
            return price
        
        # otherwise format price with currency conversions, symbol etc
        price = float(price) * self.config['PRICE_SALE'] / 100
        price = float(price) * self.config['PRICE_SCALEFACTOR']
        price = self.roundPriceToNearest(price)
        decimals = str(self.config['PRICE_TONEAREST']).split(".")
        digits = len(decimals[1]) if len(decimals) == 2 else 0 # number of digits after decimal point (for formatting reasons)
        price = round(price, digits)
        return "%s%.*f%s"%(self.config['PRICE_PREFIX'], digits, price, self.config['PRICE_SUFFIX'])
    
    def endWithMessage(self):
        
        ### Print to console that the processing has been complete
        
        IPython.display.clear_output()
        print("Items processed successfully in:")
        print(os.path.join(os.getcwd(), self.output_path))
    
    def roundPriceToNearest(self, price):
        
        ### Helper function to round a number x to the nearest 1, 0.05, etc
        
        return ceil(float(price) * (1.0 / self.config['PRICE_TONEAREST'])) * self.config['PRICE_TONEAREST']
         
    def haltProcessing(self, msg=""):
        
        ### Wait for user input before continuing
        
        return raw_input(msg)

In [ ]:
# This runs the whole process
# It is possible to change the configuration by passing parameters (see __init__ in class above for documentation)

_ = ProcessItems(
     SOLD_SHOW=False,
     NFS_SHOW=False
    )